In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import kagglehub

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version of your dataset
path = kagglehub.dataset_download("porhayrouen/aircraft-noise-report-sfo")

print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/Cleaned_Aircraft_Noise_Data.csv")
df.shape
df.head()


df = df.dropna()

Path to dataset files: /Users/macbookpro/.cache/kagglehub/datasets/porhayrouen/aircraft-noise-report-sfo/versions/1


/var/folders/wx/0m6_v4n9285892l822ksy8740000gn/T/ipykernel_86829/3218187702.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/Cleaned_Aircraft_Noise_Data.csv")


In [ ]:
df.head()


,disturbance_date_time,reporter_city,reporter_postal_code,aircraft_pca_altitude,airport_id,operation_type,aircraft_type,hour
0,2024-08-18 19:21:21,Palo Alto,94301,4911.0,SFO,A,A319,19
1,2024-08-18 19:21:42,Portola Valley,94028,4610.0,SFO,A,B739,19
2,2024-08-18 19:21:46,Palo Alto,94303,4803.0,SFO,A,A319,19
3,2024-08-18 19:24:40,Redwood City,94061,4390.0,SFO,A,B39M,19
4,2024-09-13 19:09:59,Santa Cruz,95065,11378.0,SFO,A,B739,19


In [11]:
df['reporter_city'].nunique()

73